In [1]:
import torch

In [2]:
t1 = torch.tensor([[[1,2,3],
                    [4,5,6]],
                   [[10,11,12],
                    [13,14,15]]])

In [3]:
t1.shape

torch.Size([2, 2, 3])

In [4]:
t2 = torch.tensor([[[-99,81,101],
                    [-4,54,75]],
                   [[108,-11,-12],
                    [133,144,150]]])

In [5]:
t2.shape

torch.Size([2, 2, 3])

In [6]:
t3 = torch.tensor(4.)

In [7]:
t3.shape

torch.Size([])

### Tensor can do arithmetic operations directly. That's the strength of tensor over pandas

In [8]:
y = t1*t2 +t3
y

tensor([[[ -95.,  166.,  307.],
         [ -12.,  274.,  454.]],

        [[1084., -117., -140.],
         [1733., 2020., 2254.]]])

In [9]:
print(y.grad)

None


### In torch.tensor you cannot make string array. It supports only numerical data, float. 

In [10]:
t4 = torch.tensor(['Hello','OK'])
t4

ValueError: too many dimensions 'str'

In [19]:
import numpy as np


In [20]:
#Input (temp,rainfall,humidity)
inputs = np.array([[73.,67.,43.],
                    [91.,88.,64.],
                    [87.,134.,58.],
                    [102.,43.,37.],
                    [69.,96.,70.]],dtype="float32")


In [21]:
#Targets (apple, orange)
targets = np.array([[56.,70.],
                   [81.,101.],
                   [119.,133.],
                    [22.,37.],
                    [103.,119.]], dtype="float32")

In [22]:
input_tensor = torch.from_numpy(inputs)
target_tensor = torch.from_numpy(targets)
print(input_tensor)
print(target_tensor)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [23]:
input_tensor.shape

torch.Size([5, 3])

In [24]:
target_tensor.shape

torch.Size([5, 2])

 ### How linear algebra and cost function plays in linear regression
 y_orange = w11 * x1 + w12 * x2 + w13 *x3 +b1
 
 
 
 y_apple = w21 * x1 + w22 * x2 + w23 * x3 +b2
 
 y=AX+B
 
 
 A = [w11,w12,w13
   
    w21,w22,w23]
 
 
 X = [X1,
 
    X2,
      
    X3]
      
      
      
B = [b1,

    b2]
     
     
This is how linear regression works 

In [25]:
w = torch.randn(2,3,requires_grad=True)
b = torch.randn(2,requires_grad=True)
print(w)
print(b)

tensor([[ 1.4926,  0.1573, -0.3251],
        [ 0.3469,  1.3503,  0.5836]], requires_grad=True)
tensor([-0.3269, -1.4682], requires_grad=True)


### Model creation

    @ represent matrix multiplication
    w.t() represent the transpose of a given matrix

In [26]:
def model(x):
    return x @ w.t() + b

In [27]:

preds = model(input_tensor)
print(preds)

tensor([[105.1927, 139.4232],
        [128.5359, 186.2803],
        [131.7514, 243.5040],
        [146.6535, 113.5755],
        [ 95.0066, 192.9519]], grad_fn=<AddBackward0>)


In [28]:
print(target_tensor)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [29]:
dif = preds - target_tensor
torch.sum(dif*dif)/dif.numel()

tensor(5608.0693, grad_fn=<DivBackward0>)

Mean Square Error = 11270.5098

In [30]:
def mse(t1,t2):
    dif = t1 - t2
    return torch.sum(dif*dif) / dif.numel()

Computing mse using the functions

In [31]:
#Compute loss
loss = mse(preds,target_tensor)
print(loss)

tensor(5608.0693, grad_fn=<DivBackward0>)


In [32]:
#Compute gradients
loss.backward()

In [33]:
print(w)
print(w.grad)

tensor([[ 1.4926,  0.1573, -0.3251],
        [ 0.3469,  1.3503,  0.5836]], requires_grad=True)
tensor([[4237.8618, 2756.0964, 1989.9603],
        [7071.1255, 7471.1372, 4572.4590]])


In [34]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(w)
print(b.grad)
print(b)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[ 1.4926,  0.1573, -0.3251],
        [ 0.3469,  1.3503,  0.5836]], requires_grad=True)
tensor([0., 0.])
tensor([-0.3269, -1.4682], requires_grad=True)


## Train the model using gradient descent

As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can _train_ the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [35]:
preds = model(input_tensor)
print(preds)

tensor([[105.1927, 139.4232],
        [128.5359, 186.2803],
        [131.7514, 243.5040],
        [146.6535, 113.5755],
        [ 95.0066, 192.9519]], grad_fn=<AddBackward0>)


In [36]:
loss = mse(preds, target_tensor)
loss

tensor(5608.0693, grad_fn=<DivBackward0>)

In [37]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[4237.8618, 2756.0964, 1989.9603],
        [7071.1255, 7471.1372, 4572.4590]])
tensor([45.2280, 83.1470])


In [38]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [79]:
for i in range(100):
    preds = model(input_tensor)
    loss = mse(preds, target_tensor)
    print(loss)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

tensor(0.5346, grad_fn=<DivBackward0>)
tensor(0.5345, grad_fn=<DivBackward0>)
tensor(0.5344, grad_fn=<DivBackward0>)
tensor(0.5343, grad_fn=<DivBackward0>)
tensor(0.5343, grad_fn=<DivBackward0>)
tensor(0.5342, grad_fn=<DivBackward0>)
tensor(0.5341, grad_fn=<DivBackward0>)
tensor(0.5340, grad_fn=<DivBackward0>)
tensor(0.5340, grad_fn=<DivBackward0>)
tensor(0.5339, grad_fn=<DivBackward0>)
tensor(0.5338, grad_fn=<DivBackward0>)
tensor(0.5337, grad_fn=<DivBackward0>)
tensor(0.5336, grad_fn=<DivBackward0>)
tensor(0.5336, grad_fn=<DivBackward0>)
tensor(0.5335, grad_fn=<DivBackward0>)
tensor(0.5334, grad_fn=<DivBackward0>)
tensor(0.5333, grad_fn=<DivBackward0>)
tensor(0.5333, grad_fn=<DivBackward0>)
tensor(0.5332, grad_fn=<DivBackward0>)
tensor(0.5331, grad_fn=<DivBackward0>)
tensor(0.5330, grad_fn=<DivBackward0>)
tensor(0.5330, grad_fn=<DivBackward0>)
tensor(0.5329, grad_fn=<DivBackward0>)
tensor(0.5328, grad_fn=<DivBackward0>)
tensor(0.5327, grad_fn=<DivBackward0>)
tensor(0.5327, grad_fn=<D

### Now implementing linear regression using pytorch's inbuilt function

In [81]:
import torch.nn as nn

In [82]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [83]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

### To load the data we use Tensor dataset which allows you access to rows of inputs and targets as tuples 

In [84]:
from torch.utils.data import TensorDataset

In [86]:
train_ds = TensorDataset(inputs,targets)
train_ds[:4]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.]]))

### DataLoader is used to load the data into mini batches

In [87]:
from torch.utils.data import DataLoader

In [88]:
batch_size = 5
train_dl = DataLoader(train_ds,batch_size,shuffle = True)


In [90]:
for x,y in train_dl:
    print(x)
    print(y)
    print('\n')

tensor([[ 74.,  66.,  43.],
        [103.,  43.,  36.],
        [ 73.,  67.,  43.],
        [ 68.,  96.,  71.],
        [101.,  44.,  37.]])
tensor([[ 57.,  69.],
        [ 20.,  38.],
        [ 56.,  70.],
        [104., 118.],
        [ 21.,  38.]])


tensor([[73., 66., 44.],
        [68., 97., 70.],
        [92., 87., 64.],
        [69., 96., 70.],
        [91., 88., 64.]])
tensor([[ 57.,  69.],
        [102., 120.],
        [ 82., 100.],
        [103., 119.],
        [ 81., 101.]])


tensor([[102.,  43.,  37.],
        [ 87., 135.,  57.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [ 87., 134.,  58.]])
tensor([[ 22.,  37.],
        [118., 134.],
        [ 80., 102.],
        [118., 132.],
        [119., 133.]])




### Using nn.Linear you can directly create a linear regression model object instead of manually creating a model function each and every time

In [92]:
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.5540,  0.4187, -0.3194],
        [ 0.5036,  0.4312, -0.3408]], requires_grad=True)
Parameter containing:
tensor([0.0283, 0.5620], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [93]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.5540,  0.4187, -0.3194],
         [ 0.5036,  0.4312, -0.3408]], requires_grad=True),
 Parameter containing:
 tensor([0.0283, 0.5620], requires_grad=True)]

In [94]:
preds = model(inputs)
preds

tensor([[-26.0937,  51.5618],
        [-33.9799,  62.5253],
        [-10.5879,  82.3892],
        [-50.2917,  57.8637],
        [-20.3589,  52.8501],
        [-27.0664,  51.6342],
        [-34.7180,  61.7533],
        [-11.4613,  82.5520],
        [-49.3190,  57.7912],
        [-20.1243,  52.0057],
        [-26.8318,  50.7898],
        [-34.9526,  62.5978],
        [ -9.8499,  83.1611],
        [-50.5263,  58.7081],
        [-19.3862,  52.7777]], grad_fn=<AddmmBackward0>)

## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [95]:
import torch.nn.functional as F
loss_fn = F.mse_loss

In [96]:
loss = loss_fn(model(inputs),targets)
loss

tensor(6628.8115, grad_fn=<MseLossBackward0>)

## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term _stochastic_ indicates that samples are selected in random batches instead of as a single group.

In [97]:
# Defining optimizer
opt = torch.optim.SGD(model.parameters(),lr=1e-5)

In [100]:
def fit(num_epochs,model,loss_fn,opt,train_dl):
    #Repeat for given epochs
    for i in range(num_epochs):
        #Train with the batches of data
        for x,y in train_dl:
            #Generate prediction
            pred = model(x)
            #Calculate loss
            loss = loss_fn(pred,y)
            #Compute gradients
            loss.backward()
            #update parameters using gradients
            opt.step()
            #reset the gradient to zero
            opt.zero_grad()
        print('Epoch [{}/{}], Loss: {:.4f}'.format(i+1, num_epochs, loss.item()))
fit(100,model,loss_fn,opt,train_dl)
            

Epoch [1/100], Loss: 1651.3601
Epoch [2/100], Loss: 517.9539
Epoch [3/100], Loss: 582.2565
Epoch [4/100], Loss: 390.6573
Epoch [5/100], Loss: 28.5129
Epoch [6/100], Loss: 444.6581
Epoch [7/100], Loss: 424.7068
Epoch [8/100], Loss: 257.3707
Epoch [9/100], Loss: 126.6385
Epoch [10/100], Loss: 238.9079
Epoch [11/100], Loss: 466.9403
Epoch [12/100], Loss: 372.1020
Epoch [13/100], Loss: 235.8814
Epoch [14/100], Loss: 512.0754
Epoch [15/100], Loss: 245.7769
Epoch [16/100], Loss: 241.6556
Epoch [17/100], Loss: 369.8845
Epoch [18/100], Loss: 295.5966
Epoch [19/100], Loss: 109.0165
Epoch [20/100], Loss: 307.4822
Epoch [21/100], Loss: 211.9629
Epoch [22/100], Loss: 130.9624
Epoch [23/100], Loss: 241.7461
Epoch [24/100], Loss: 297.9491
Epoch [25/100], Loss: 198.4643
Epoch [26/100], Loss: 269.3156
Epoch [27/100], Loss: 131.0230
Epoch [28/100], Loss: 19.2312
Epoch [29/100], Loss: 176.7535
Epoch [30/100], Loss: 91.4471
Epoch [31/100], Loss: 234.4617
Epoch [32/100], Loss: 27.8532
Epoch [33/100], Loss

In [101]:
preds = model(inputs)
preds

tensor([[ 57.7045,  72.2765],
        [ 78.5700,  95.5735],
        [125.1981, 141.6810],
        [ 24.4947,  47.0259],
        [ 93.6733, 104.5189],
        [ 56.4256,  71.2520],
        [ 77.7539,  94.6613],
        [125.1372, 141.7742],
        [ 25.7737,  48.0503],
        [ 94.1361, 104.6311],
        [ 56.8884,  71.3643],
        [ 77.2911,  94.5490],
        [126.0142, 142.5933],
        [ 24.0319,  46.9136],
        [ 94.9522, 105.5433]], grad_fn=<AddmmBackward0>)

In [102]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])